# Clase 8: Métricas Avanzadas y Decisión (Proyecto Final)

Llegamos al final. Tenemos Datos, Información y Conocimiento. Ahora necesitamos **Sabiduría**.
Actuarás como Gerente General (GM) de un equipo con presupuesto limitado.

## Objetivo
- Calcular métricas avanzadas (Eficiencia).
- Tomar decisiones de fichaje basadas en "Valor por Dinero" (Moneyball).

---

In [ ]:
from google.colab import files
import os
import pandas as pd

# Setup Kaggle rápido
if not os.path.exists("baseball_data"):
    if not os.path.exists("kaggle.json"):
        try: files.upload()
        except: pass
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/ 2>/dev/null
    !chmod 600 ~/.kaggle/kaggle.json 2>/dev/null
    !kaggle datasets download -d open-source-sports/baseball-databank
    !unzip -q baseball-databank.zip -d baseball_data

# Cargamos datos
batting = pd.read_csv('baseball_data/Batting.csv')
salaries = pd.read_csv('baseball_data/Salaries.csv')
people = pd.read_csv('baseball_data/People.csv') # Para tener nombres reales

print("Datos cargados.")

## 1. Definir Métrica de Valor
Vamos a usar **OBP (On-Base Percentage)** como nuestra métrica estrella, ya que "embasarse" correlaciona alto con ganar juegos.

$$ OBP = \frac{H + BB + HBP}{AB + BB + HBP + SF} $$

*Nota: Para simplificar, usaremos una aproximación con las columnas disponibles.*

In [ ]:
# Filtramos año reciente (ej. 2015)
df_2015 = batting[batting['yearID'] == 2015].copy()

# Calculamos OBP simplificado
# H = Hits, BB = Base on Balls, HBP = Hit By Pitch, AB = At Bats, SF = Sacrifice Flies
df_2015['OBP'] = (df_2015['H'] + df_2015['BB'] + df_2015['HBP']) / (df_2015['AB'] + df_2015['BB'] + df_2015['HBP'] + df_2015['SF'])

# Filtramos jugadores con pocos turnos al bat (ruido estadístico)
df_2015 = df_2015[df_2015['AB'] > 400]

print("Métricas calculadas.")

## 2. Cruzar con Salarios
Queremos jugadores con **Alto OBP** pero **Bajo Salario**.

In [ ]:
salaries_2015 = salaries[salaries['yearID'] == 2015]

# Unimos tablas
df_final = pd.merge(df_2015, salaries_2015, on=['playerID', 'yearID', 'teamID'])

# Unimos con nombres
df_final = pd.merge(df_final, people[['playerID', 'nameFirst', 'nameLast']], on='playerID')

# Calculamos "Costo por punto de OBP" (Eficiencia)
# Entre menor sea este número, más barato nos sale que el jugador se embase
df_final['Efficiency'] = df_final['salary'] / df_final['OBP']

df_final[['nameFirst', 'nameLast', 'salary', 'OBP', 'Efficiency']].sort_values('Efficiency').head(10)

## 3. La Decisión (El Fichaje)
Arriba tienes la lista de las "Gemas Ocultas" del 2015. Jugadores que aportaron mucho ofensivamente cobrando "poco".

**Ejercicio Final:**
Si tuvieras 10 millones de dólares de presupuesto, ¿a qué 3 jugadores de esta lista ficharías para maximizar tu OBP total?